<a href="https://colab.research.google.com/github/ajeet-trader/FX-bot/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import streamlit as st
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import ta

# ------------------- APP TITLE --------------------
st.set_page_config(page_title="Forex Signal Bot", layout="centered")
st.title("📈 Forex Smart Signal Bot")
st.markdown("Made by Ajeet – Strategy: EMA + RSI + Candle Reversal")

# ------------------- SIDEBAR INPUT --------------------
st.sidebar.header("🛠️ Settings")

symbols = {
    "EURUSD": "EURUSD=X",
    "GBPUSD": "GBPUSD=X",
    "USDJPY": "JPY=X",
    "AUDUSD": "AUDUSD=X",
    "USDCAD": "CAD=X",
    "USDCHF": "CHF=X",
    "NZDUSD": "NZDUSD=X"
}

symbol_name = st.sidebar.selectbox("Select Symbol", list(symbols.keys()))
symbol = symbols[symbol_name]

timeframe = st.sidebar.selectbox("Select Timeframe", ["1h", "30m", "15m"])
period = st.sidebar.selectbox("Data Period", ["90d", "180d", "730d"])

# ------------------- FETCH DATA --------------------
@st.cache_data(ttl=600)
def fetch_data(symbol, period, interval):
    df = yf.download(tickers=symbol, period=period, interval=interval, progress=False)
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    return df.dropna()

df = fetch_data(symbol, period, timeframe)

# ------------------- CALCULATE SIGNALS --------------------
df["ema50"] = ta.trend.ema_indicator(df["Close"], window=50)
df["ema200"] = ta.trend.ema_indicator(df["Close"], window=200)
df["rsi"] = ta.momentum.rsi(df["Close"], window=14)

def generate_signals(df):
    signals = []
    for i in range(1, len(df)):
        row, prev = df.iloc[i], df.iloc[i-1]
        if (
            row["ema50"] > row["ema200"] and
            row["rsi"] < 30 and
            row["Close"] > row["Open"] and
            prev["Close"] < prev["Open"]
        ):
            entry = row["Close"]
            sl = row["Low"] - 0.0010
            tp = entry + 2 * (entry - sl)
            signals.append({"index": i, "signal": "Buy", "entry": entry, "sl": sl, "tp": tp})
        elif (
            row["ema50"] < row["ema200"] and
            row["rsi"] > 70 and
            row["Close"] < row["Open"] and
            prev["Close"] > prev["Open"]
        ):
            entry = row["Close"]
            sl = row["High"] + 0.0010
            tp = entry - 2 * (sl - entry)
            signals.append({"index": i, "signal": "Sell", "entry": entry, "sl": sl, "tp": tp})
    return signals

signals = generate_signals(df)

# ------------------- PLOT LAST SIGNAL --------------------
if signals:
    last = signals[-1]
    idx = last["index"]
    window = df.iloc[max(0, idx-30): idx+5]

    st.success(f"**{last['signal']} Signal Found** at {df.index[idx]}")
    st.write(f"**Entry:** {last['entry']:.5f} | **TP:** {last['tp']:.5f} | **SL:** {last['sl']:.5f}")

    fig = go.Figure([
        go.Candlestick(
            x=window.index,
            open=window["Open"],
            high=window["High"],
            low=window["Low"],
            close=window["Close"],
            name="Price"
        ),
        go.Scatter(x=[df.index[idx]], y=[last["entry"]],
                   mode="markers+text", marker=dict(color="blue", size=10),
                   name="Entry", text=["Entry"], textposition="bottom center"),
        go.Scatter(x=[df.index[idx]], y=[last["tp"]],
                   mode="markers+text", marker=dict(color="green", size=10),
                   name="TP", text=["TP"], textposition="top right"),
        go.Scatter(x=[df.index[idx]], y=[last["sl"]],
                   mode="markers+text", marker=dict(color="red", size=10),
                   name="SL", text=["SL"], textposition="bottom right")
    ])
    fig.update_layout(title=f"{symbol_name} | {last['signal']} Signal")
    st.plotly_chart(fig, use_container_width=True)
else:
    st.warning("No valid signals found in selected period.")

# ------------------- FOOTER --------------------
st.caption("⚡ Built in Streamlit | v0.1")


SyntaxError: invalid syntax (ipython-input-1-1649162778.py, line 2)